In [1]:
import pandas as pd

# 데이터 읽기
df = pd.read_csv('Mall_Customers.csv')

# 데이터 구조 보기
print(df.head())

   CustomerID  Gender  Age  Annual Income (k$)  Spending Score (1-100)
0           1    Male   19                  15                      39
1           2    Male   21                  15                      81
2           3  Female   20                  16                       6
3           4  Female   23                  16                      77
4           5  Female   31                  17                      40


In [2]:
#결측치 처리
print(df.isnull().sum())

CustomerID                0
Gender                    0
Age                       0
Annual Income (k$)        0
Spending Score (1-100)    0
dtype: int64


In [5]:
#엔코딩
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

df['Gender'] = le.fit_transform(df['Gender'])       # female=0, male=1

In [6]:
print(df.head())

   CustomerID  Gender  Age  Annual Income (k$)  Spending Score (1-100)
0           1       1   19                  15                      39
1           2       1   21                  15                      81
2           3       0   20                  16                       6
3           4       0   23                  16                      77
4           5       0   31                  17                      40


In [9]:
from sklearn.model_selection import train_test_split

X = df.drop(['Annual Income (k$)', "Spending Score (1-100)"], axis=1)
y = df['Spending Score (1-100)']

In [10]:
from sklearn.model_selection import train_test_split

# 다시 train/test 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
import torch

X_train = torch.tensor(X_train.values, dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

In [12]:
import torch.nn as nn

class RegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 16)  # 은닉층
        self.fc2 = nn.Linear(16, 1)                  # 출력층
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

model = RegressionModel()

In [13]:
# 📍 Step 4: 손실 함수 및 옵티마이저 설정 (회귀용)
import torch.optim as optim

criterion = nn.MSELoss()  # 회귀용 손실 함수
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam 옵티마이저

In [ ]:
epochs = 100000

for epoch in range(epochs):
    model.train()
    
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 30 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [30/1000000], Loss: 420.1230
Epoch [60/1000000], Loss: 420.0914
Epoch [90/1000000], Loss: 420.1482
Epoch [120/1000000], Loss: 420.1004
Epoch [150/1000000], Loss: 420.1285
Epoch [180/1000000], Loss: 420.1322
Epoch [210/1000000], Loss: 420.1267
Epoch [240/1000000], Loss: 420.0934
Epoch [270/1000000], Loss: 420.1396
Epoch [300/1000000], Loss: 420.0875
Epoch [330/1000000], Loss: 420.1613
Epoch [360/1000000], Loss: 420.0876
Epoch [390/1000000], Loss: 420.1353
Epoch [420/1000000], Loss: 420.1344
Epoch [450/1000000], Loss: 420.1261
Epoch [480/1000000], Loss: 420.2086
Epoch [510/1000000], Loss: 420.1587
Epoch [540/1000000], Loss: 420.1851
Epoch [570/1000000], Loss: 420.1305
Epoch [600/1000000], Loss: 420.1111
Epoch [630/1000000], Loss: 420.1035
Epoch [660/1000000], Loss: 420.1363
Epoch [690/1000000], Loss: 420.1091
Epoch [720/1000000], Loss: 420.1263
Epoch [750/1000000], Loss: 420.0988
Epoch [780/1000000], Loss: 420.1710
Epoch [810/1000000], Loss: 420.1765
Epoch [840/1000000], Loss: 420.

KeyboardInterrupt: 

In [20]:
# 손실함수 MSE 
model.eval()
with torch.no_grad():
    preds = model(X_test)
    mse = criterion(preds, y_test)
    print(f'Test MSE: {mse.item():.4f}')

Test MSE: 465.8254


In [16]:
#  R2 Score 평가
from sklearn.metrics import r2_score

# numpy로 변환해서 r2_score 계산
r2 = r2_score(y_test.numpy(), preds.numpy())
print(f'Test R2 Score: {r2:.4f}')

Test R2 Score: -1.0445
